In [1]:
import os
import pandas as pd
import csv
import numpy as np

In [5]:
# Script for changing the header of each CSV, and removing the last 3 columns (not needed right now)

def modify_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Drop the last three columns
    df = df.iloc[:, :-3]

    # Update the header row
    df.columns = ['Time', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Bx', 'By', 'Bz']

    # Change the first column to be "0, 0.5, 1.0, 1.5, 2, 2.5..."
    df['Time'] = [0.5 * i for i in range(len(df))]

    # Save the modified DataFrame back to the same CSV file
    df.to_csv(file_path, index=False)

def modify_all_csvs_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            modify_csv(file_path)

# Example usage:
directory_path = './12pm_LTDN'
modify_all_csvs_in_directory(directory_path)

In [8]:
# Script for changing the file from being every 0.5s to every 1s

def delete_every_other_row(input_csv):
    # Define the output CSV filename
    base, ext = os.path.splitext(input_csv)
    output_csv = f"{base}05{ext}"

    # Read the input CSV file
    with open(input_csv, 'r', newline='') as infile:
        reader = csv.reader(infile)
        rows = list(reader)

    # Prepare to write to the output CSV file
    with open(output_csv, 'w', newline='') as outfile:
        writer = csv.writer(outfile)

        # Write the header row
        writer.writerow(rows[0])

        # Write the first data row
        writer.writerow(rows[1])

        # Delete every other row starting with the third line
        for i in range(2, len(rows)):
            if (i - 2) % 2 == 1:
                writer.writerow(rows[i])

    print(f"Processed CSV saved as: {output_csv}")

def modify_all_csvs_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            delete_every_other_row(file_path)

# Example usage:
directory_path = './12pm_LTDN'
modify_all_csvs_in_directory(directory_path)

Processed CSV saved as: ./12pm_LTDN\thompson05.csv
Processed CSV saved as: ./12pm_LTDN\yaw05.csv


In [2]:
# Script for taking the 0.5s file and using lininterp to make it every 0.01s

def interpolate_csv(input_csv):
    # Read the input CSV file
    df = pd.read_csv(input_csv)

    # Extract the time and data columns
    time = df.iloc[:, 0].values
    data = df.iloc[:, 1:].values

    # Create new time values from the minimum to maximum time at 0.01 second intervals
    new_time = np.arange(time[0], time[-1] + 0.01, 0.01)

    # Perform linear interpolation for each data column
    new_data = np.zeros((len(new_time), data.shape[1]))
    for i in range(data.shape[1]):
        new_data[:, i] = np.interp(new_time, time, data[:, i])

    # Create a new DataFrame with the interpolated data
    new_df = pd.DataFrame(data=np.column_stack((new_time, new_data)), columns=df.columns)

    # Define the output CSV filename
    base, ext = input_csv.rsplit('.', 1)
    output_csv = f"{base}_interpolated.{ext}"

    # Save the new DataFrame to the output CSV file
    new_df.to_csv(output_csv, index=False)
    print(f"Interpolated CSV saved as: {output_csv}")

def modify_all_csvs_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            interpolate_csv(file_path)

# Example usage:
directory_path = './csvs'
modify_all_csvs_in_directory(directory_path)

Interpolated CSV saved as: ./csvs\001_3am_interpolated.csv
Interpolated CSV saved as: ./csvs\001_6am_interpolated.csv
